<a href="https://colab.research.google.com/github/adimis-ai/Multiple-Document-Abstractive-Long-Form-Q-A/blob/main/Multiple_Document_Abstractive_Long_Form_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

In [ ]:
# setup logs
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
#import faiss document store and assign it to document_store variable
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str="Flat",return_embedding=True)

In [ ]:
#imports for working with data
from haystack.utils import convert_files_to_docs, clean_wiki_text

# Let's first get some files that we want to use
doc_dir = "gdrive/MyDrive/wiki_gameofthrones_txt1/"

# Convert files to dicts
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

In [ ]:
#import dense passage retriever
from haystack.nodes import DensePassageRetriever

# retriever takes in query & convert it to dense vectors using pretrained model for generator
# search & fetches relevant passage(context) from the document store & convert it to dense vectors using pretrained model for generator
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="vblagoje/dpr-question_encoder-single-lfqa-wiki",
    passage_embedding_model="vblagoje/dpr-ctx_encoder-single-lfqa-wiki",
)

document_store.update_embeddings(retriever)

In [ ]:
# we now initalize our reader/generator.
from haystack.nodes import Seq2SeqGenerator

# Here we use a Seq2SeqGenerator with the vblagoje/bart_lfqa model
generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

In [ ]:
# With a Haystack Pipeline you can stick together your building blocks to a search pipeline.
# Under the hood, Pipelines are Directed Acyclic Graphs (DAGs) that you can easily customize for your own use cases
from haystack.pipelines import GenerativeQAPipeline

#GenerativeQAPipeline that combines a retriever and a reader/generator to answer our questions.
pipe = GenerativeQAPipeline(generator, retriever)

In [ ]:
from haystack.utils import print_answers

def ans(QUERY):
  output = pipe.run(query=QUERY, params={"Retriever": {"top_k": 3}})
  cleaned_output = print_answers(output, details="minimum")
  return cleaned_output

In [ ]:
Question = input()
Answer = ans(Question)